# Fetch sales order with items

In [13]:
from app.core.models import SalesFlatOrder
import dateutil
orders = SalesFlatOrder.objects.filter(
    store=10,
    status='complete',
    increment_id__in=[]
    created_at__range=(
        dateutil.parser.parse("2017-10-03"),
        dateutil.parser.parse("2017-10-04")))

/Users/varunpasad/django/lib/python2.7/site-packages/django/db/models/fields/__init__.py:1451: RuntimeWarning: DateTimeField SalesFlatOrder.created_at received a naive datetime (2017-10-04 00:00:00) while time zone support is active.
  RuntimeWarning)


In [8]:
for sale_order in orders:
    for item in sale_order.items.all():
        print item.sku

SF_SANKARA
SF_SEER
SF_SEER
SF_SEER
SF_MATHI
SF_LADY_FISH
SF_PRAWN
SF_ANCHOVY 
SF_SEER
SF_CATLA
SF_KARAI_PODI
SF_KARAI_PODI
SF_PRAWN
SF_SEER
SF_BABY_SEER
SF_SEER
SF_FWPOMFRET
SF_PRAWN
SF_SANKARA
SF_SEER_CURRY
SF_BLUE_CRAB
SF_ANCHOVY 
SF_BABY_SEER


In [14]:
from app.core.lib import user_controller
from app.core.lib.user_controller import CustomerSearchController
params_data = []
for order in orders:
    shipping_address = order.shipping_address.all()[0]
    user = CustomerSearchController.load_basic_info(order.customer_id)
    params = {
        "increment_id": order.increment_id, 
        "created_at": str(order.created_at),
        "total_amount": float(order.grand_total),
        "subtotal_amount": float(order.subtotal),
        "shipping_amount": float(order.shipping_amount),
        "discount_amount": float(order.discount_amount),
        "discount_description": order.discount_description or '',
        "payment_method": order.payment.all()[0].method,
        "medium": order.medium,

        "customer": {
            "partner_name": order.customer_firstname,
            "phone": user[2],
            "email": order.customer_email
        },
        "shipping_address": {
            "street": shipping_address.street,
            "street2": shipping_address.region or "",
            "city": shipping_address.city,
        },
        "store":  order.store.code

    }

    for item in order.items.all():
        params.setdefault("product_list", []).append({
            "sku": item.sku.strip(),
            "ordered_qty": float(item.qty_ordered),
            "weight": float(item.weight),
            "unit_price": float(item.price),
            "total_amount": float(item.row_total),
            "discount_amount": float(item.discount_amount or 0)

        })
    params_data.append(params)

# Connect to odoo

In [33]:
import xmlrpclib
import json

username = 'varun@tendercuts.in' #the user
pwd = 'qwerty123'      #the password of the user
dbname = 'tcuts'    #the database


# Get the uid
sock_common = xmlrpclib.ServerProxy ('http://localhost:8069/xmlrpc/common')
uid = sock_common.login(dbname, username, pwd)

# replace localhost with the address of the server
sock = xmlrpclib.ServerProxy('http://localhost:8069/xmlrpc/object')


for param in params_data:
    result = sock.execute_kw(
        dbname,
        uid,
        pwd,
        'tcuts.sync.record',
        'create',
        [{
            "increment_id": param['increment_id'],
            "payload": json.dumps(param)
        }]
        )
    print (param)
    result = sock.execute_kw( dbname, uid, pwd,
        'tcuts.sync.record', 'action_sync',
        [[result]])

{'medium': 4L, 'discount_description': '', 'payment_method': u'cashforpos', 'total_amount': 730.35, 'discount_amount': 0.0, 'customer': {'partner_name': u'susil', 'phone': u'9244103812', 'email': u'9244103812@email.com'}, 'increment_id': u'WP271506979637417', 'created_at': '2017-10-03 02:57:17+00:00', 'shipping_amount': 0.0, 'product_list': [{'sku': u'MRT_LEMONE_CHICKEN', 'total_amount': 382.95, 'weight': 0.555, 'unit_price': 690.0, 'ordered_qty': 0.555, 'discount_amount': 0.0}, {'sku': u'MRT_MASRAS_CHICKEN_65_BONELESS', 'total_amount': 347.4, 'weight': 0.6, 'unit_price': 579.0, 'ordered_qty': 0.6, 'discount_amount': 0.0}], 'shipping_address': {'city': u'Valasaravakkam', 'street2': '', 'street': u'No - 47, Ground Floor'}, 'subtotal_amount': 730.35, 'store': u'valasaravakkam'}
{'medium': 4L, 'discount_description': '', 'payment_method': u'cashforpos', 'total_amount': 364.27, 'discount_amount': 0.0, 'customer': {'partner_name': u'joe', 'phone': u'9884180888', 'email': u'9884180888@email.

{'medium': 4L, 'discount_description': '', 'payment_method': u'cashforpos', 'total_amount': 139.69, 'discount_amount': 0.0, 'customer': {'partner_name': u'athafi', 'phone': u'9884690358', 'email': u'9884690358@email.com'}, 'increment_id': u'WP271507010722746', 'created_at': '2017-10-03 11:35:22+00:00', 'shipping_amount': 0.0, 'product_list': [{'sku': u'CHK_WHL_SKIN_OFF', 'total_amount': 139.69, 'weight': 0.61, 'unit_price': 229.0, 'ordered_qty': 0.61, 'discount_amount': 0.0}], 'shipping_address': {'city': u'Valasaravakkam', 'street2': '', 'street': u'No - 47, Ground Floor'}, 'subtotal_amount': 139.69, 'store': u'valasaravakkam'}
{'medium': 4L, 'discount_description': u'30 Tender Rewards', 'payment_method': u'cashforpos', 'total_amount': 97.1, 'discount_amount': -30.0, 'customer': {'partner_name': u'Siva', 'phone': u'9894510924', 'email': u'shivpikka@gmail.com'}, 'increment_id': u'WP271507012308113', 'created_at': '2017-10-03 12:01:48+00:00', 'shipping_amount': 0.0, 'product_list': [{'s

{'medium': 4L, 'discount_description': '', 'payment_method': u'cashforpos', 'total_amount': 59.54, 'discount_amount': 0.0, 'customer': {'partner_name': u'sakshi', 'phone': u'9971038869', 'email': u'9971038869@email.com'}, 'increment_id': u'WP271507020049725', 'created_at': '2017-10-03 14:10:49+00:00', 'shipping_amount': 0.0, 'product_list': [{'sku': u'CHK_WHL_SKIN_OFF', 'total_amount': 59.54, 'weight': 0.26, 'unit_price': 229.0, 'ordered_qty': 0.26, 'discount_amount': 0.0}], 'shipping_address': {'city': u'Valasaravakkam', 'street2': '', 'street': u'No - 47, Ground Floor'}, 'subtotal_amount': 59.54, 'store': u'valasaravakkam'}
{'medium': 4L, 'discount_description': '', 'payment_method': u'cashforpos', 'total_amount': 278.95, 'discount_amount': 0.0, 'customer': {'partner_name': u'Sarankumar', 'phone': u'9941120620', 'email': u'sarankumarpm@gmail.com'}, 'increment_id': u'WP271507022940447', 'created_at': '2017-10-03 14:59:00+00:00', 'shipping_amount': 0.0, 'product_list': [{'sku': u'MRT_M

In [37]:
import xmlrpclib
import json

username = 'varun@tendercuts.in' #the user
pwd = 'qwerty123'      #the password of the user
dbname = 'tcuts'    #the database


# Get the uid
sock_common = xmlrpclib.ServerProxy ('http://localhost:8069/xmlrpc/common')
uid = sock_common.login(dbname, username, pwd)

# replace localhost with the address of the server
sock = xmlrpclib.ServerProxy('http://localhost:8069/xmlrpc/object')
result = sock.execute_kw(
    dbname,
    uid,
    pwd,
    'stock.picking',
    'create',
    [{
#         'name': "Test order",
        'picking_type_id': 11,
        'location_id': 22,
        'location_dest_id': 23,
        'min_date': '2017-10-23'
    }])